In [1]:
import networkx as nx
import gurobipy as gb
from itertools import combinations, chain
#import pygraphviz as pygv
import os
from IPython.display import SVG, display

In [2]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

#
# Drawing 
# functions
#

def DrawInitialGraph():
    global DrawG
    DrawG = pygv.AGraph(undirected='true', strict='true', splines='true')


    for i in G.nodes():
        pos = str(G.node[i]['x'] * args.scale) + ',' + str((G.node[i]['y'])* args.scale)
        DrawG.add_node (i, shape='circle', pos=pos, width=0.1, fontsize=9)
    
    DrawG.layout(prog='neato', args='-n')
    DrawG.draw (path=str(basename) + '.svg', format='svg')
    

def DrawSol (x):
    for i,j in G.edges():
        lab = x[i,j].x
        G[i][j]['capacity'] = x[i,j].x
        if x[i,j].x > 0.00001:
            if x[i,j].x > 0.999999:
                DrawG.add_edge(i, j, color='black', label=lab, fontsize=9)
            else:
                DrawG.add_edge(i, j, color='red', label=lab)
    path=str(basename)+'_sol.svg'
    DrawG.draw (path=path, format='svg')
    DrawG.delete_edges_from(DrawG.edges())

In [3]:
class args:
    filename = None
    scale = 40

## An integer ﬁxed-charge multicommodity ﬂow (FCMF) model for train unit scheduling

$$
y_a = \begin{cases}
1 \text{ if arc $a$ is in the tree} \\
0 \text { otherwise}
\end{cases}
$$


\begin{alignat}{3}
\min W_{1}\sum_{k \in K} \sum_{p \in P^k}c_{p}x_{p} + W_{2}\sum_{a \in A}y_{a} &&\\
\sum_{p \in P^k}x_{p} \le b^k &&\;\; \forall k \in K\\
\sum_{k \in K_{j}} \sum_{p \in P^k_{j}} D^j_{f,k}x_{p} \le d^j_{f} &&\;\; \forall f \in \bar{F}_{j} \text{ , } \forall j \in N\\
\sum_{k \in K_{a}} \sum_{p \in P^k_{a}} x_{p} \le m_{a}y_{a} &&\;\; \forall a \in A\\
\sum_{a \in \delta^-(j)} y_{a} = 1 &&\;\; \forall j \in N^-_{B}\\
\sum_{a \in \delta^+(j)} y_{a} = 1 &&\;\; \forall j \in N^+_{B}\\
\tau^D_{arr(i)}(\sum_{a \in \delta^+(i)} y_{a} - 1) + \tau^C_{dep(j)}(\sum_{a \in \delta^-(j)} y_{a} - 1) \le e_{ij} && \;\; \forall (i,j) \in A^*\\
x_{p} \in Z_{+}, \; \forall p \in P^k, \; \forall k \in K; \; y_{a} \in \{0,1\}, \; \forall a \in A\\
\end{alignat}

### Variables are declared BINARY but the polytope is integral

In [4]:
args.filename = "mst.gml"
args.scale = 40

basename = os.path.splitext(args.filename)[0]
G = nx.read_graphml (args.filename, node_type=int)

print ("G has", G.number_of_nodes(), "nodes and", G.number_of_edges(), "edges")

G has 10 nodes and 45 edges


In [5]:
D = G.to_directed()
root = 1
#DrawInitialGraph()
#display(SVG(filename=basename+'.svg'))

In [6]:
# adding s and t as nodes
nodessetplusst = list(G.nodes())
nodessetplusst.append(11)
nodessetplusst.append(12)
#nodessetplusst

In [7]:
# need to add necessary arcs outgoing from s and ingoing to t

# also we use a path variable x as a G[u][v]['cost'] for u,v in D.edges() result, because we don't have number of units in our graph

In [8]:
# all unit types - have to be added to a structure of the graph like an extra value
K = 3

# have to calculate w1 and w2 for an objective function
w1 = 3
w2 = 2

In [9]:
fcmf = gb.Model()


x = fcmf.addVars(D.edges(), \
                        obj=[w1 * G[u][v]['cost'] for u,v in D.edges()],\
                        vtype=gb.GRB.INTEGER,\
                        name='x')

y = fcmf.addVars(D.edges(), ub=1.0, \
                         obj = w2, \
                         vtype=gb.GRB.BINARY, name='y')

f = fcmf.addVars(D.edges(), G.nodes(), \
                         ub=G.number_of_nodes(), \
                         vtype=gb.GRB.CONTINUOUS, name='f')

fcmf.update()
fcmf.write('fcmf.lp')

In [10]:
#
# Constraints with fleet size upper bound
#

# b - fleet size upper bound
b = 10000

for u,v in D.edges():
    name = "Upper_bound_"+str(u)+"_Com_"+str(v)
    fcmf.addConstr(G[u][v]['cost']  <= b, name)

fcmf.update()

#
# Coupling constraints
#

# m = maximim number of coupled units flowing through arc
# have to be added to a structure of the graph like an extra value
m = 10

for u,v in D.edges():
    for x, y in G.edges():
        name = "Upper_bound_"+str(u)+"_Com_"+str(v)
        fcmf.addConstr(G[u][v]['cost'] <= m * G[x][y]['cost'], name)
            

fcmf.update()

#
# To forbid coupling / decoupling in banned locations
#

for v, k in D.edges():
    if (v != k):
        name = "Coupling/Decoupling_"+str(v)+"_Com_"+str(k)
        fcmf.addConstr(f.sum('*',v,k) == 1, name)
        fcmf.addConstr(f.sum(v,'*',k) == 1, name)

fcmf.update()
fcmf.write('fcmf.lp')

In [11]:
fcmf.optimize()

Optimize a model with 4320 rows, 1080 columns and 1620 nonzeros
Variable types: 900 continuous, 180 integer (90 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 1e+04]
  Bounds range     [1e+00, 1e+01]
  RHS range        [1e+00, 1e+00]
Presolve removed 101 rows and 0 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [12]:
#DrawSol(x)
#display(SVG(filename=basename+'_sol.svg'))